<a href="https://colab.research.google.com/github/riotrip/ml-smt5/blob/main/TG1_JS03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Praktikum JS 03**
## **Rio Tri Prayogo/2341720236/TI 3F**

Pada tugas pratikum ini Anda akan menggunakan data "Wisconsin Breast Cancer". Data tersebut terdiri dari 569 data yang digunakan untuk mendiagnonis jenis kanker Malignant (M) dan Benign (B). Tugas Anda adalah,

1. Pisahkan antara variabel yang dapat digunakan dan variabel yang tidak dapat digunakan.
2. Lakukan proses encoding pada kolom "diagnosis".
3. Lakukan proses standardisasi pada semua kolom yang memiliki nilai numerik.
4. Lakukan proses seleksi fitur. Anda dapat menggunakan SelectKBest.
5. Lakukan proses pengujian dengan model Logistic Regression seperti pada praktikum 1.
6. Anda dapat menggunakan model pipeline untuk mempermudah perkejaan Anda.
7. Berdasarkan hasil analisa Anda, berapa jumlah fitur terbaik yang dapat digunakan? Apa saja fitur tersebut?

### Import library dan data wbc.csv

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import chi2, SelectKBest, RFE, f_classif
from sklearn.metrics import accuracy_score, classification_report

In [24]:
df = pd.read_csv("/content/drive/MyDrive/PembelajaranMesin_Rio/docs/wbc.csv")

### Memisahkan variabel yang dapat digunakan dan variabel yang tidak dapat digunakan.

Pemisahan menggunakan iloc, dengan mengambil index variabel yang dapat digunakan saja. Variabel 'id' dan 'Unnamed: 32' tidak digunakan dan bertepatan diposisi awal dan akhir sehingga diisi 1:-1 pada bagian iloc

In [25]:
X = df.iloc[:, 1:-1]
y = df['diagnosis']

### Melakukan proses encoding pada kolom "diagnosis".

 Encoding kolom diagnosis yang semula berisi label berupa huruf M (Malignant) dan B (Benign), akan diubah menjadi nilai numerik berupa 1 untuk M dan 0 untuk B.

In [26]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

### Melakukan proses standardisasi pada semua kolom yang memiliki nilai numerik.

Function select_dtypes untuk memastikan hanya kolom numerik yang diproses, yaitu kolom dengan tipe data float64 dan int64. Lalu akan disimpan di variabel num_cols yang nantinya akan di standarisasi menggunakan fit_transform. Penggunaan df2 berfungsi untuk membuat sebuah copy() dari data sehingga tidak akan mengubah data asli.

In [27]:
num_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

In [28]:
num_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [29]:
preprocess = ColumnTransformer([
    ("num", num_tf, num_cols),
])

In [55]:
selector_filter = SelectKBest(score_func=f_classif, k=9)

pipe_filter = Pipeline([
    ("prep", preprocess),
    ("sel", selector_filter),
    ("clf", LogisticRegression(max_iter=1000, random_state=42))
])

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, stratify=y_encoded, test_size=0.2, random_state=42)

# Fit dan evaluasi model
pipe_filter.fit(X_train, y_train)
pred = pipe_filter.predict(X_test)

print("=== Wisconsin Breast Cancer Dataset ===")
print("=== Filter (ANOVA) + Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred, target_names=['Benign (B)', 'Malignant (M)']))

=== Wisconsin Breast Cancer Dataset ===
=== Filter (ANOVA) + Logistic Regression ===
Accuracy: 0.9736842105263158
               precision    recall  f1-score   support

   Benign (B)       0.97      0.99      0.98        72
Malignant (M)       0.98      0.95      0.96        42

     accuracy                           0.97       114
    macro avg       0.97      0.97      0.97       114
 weighted avg       0.97      0.97      0.97       114



In [58]:
# 1) Nama fitur setelah preprocess
feat_names = pipe_filter.named_steps["prep"].get_feature_names_out()
print("Nama fitur:", feat_names)
print("\n")

# 2) Mask & skor fitur terpilih (SelectKBest)
sel = pipe_filter.named_steps["sel"]
mask = sel.get_support()
selected_names = feat_names[mask]
selected_scores = sel.scores_[mask]
top = sorted(zip(selected_names, selected_scores), key=lambda t: t[1], reverse=True)
print("Top fitur:", top)

Nama fitur: ['num__radius_mean' 'num__texture_mean' 'num__perimeter_mean'
 'num__area_mean' 'num__smoothness_mean' 'num__compactness_mean'
 'num__concavity_mean' 'num__concave points_mean' 'num__symmetry_mean'
 'num__fractal_dimension_mean' 'num__radius_se' 'num__texture_se'
 'num__perimeter_se' 'num__area_se' 'num__smoothness_se'
 'num__compactness_se' 'num__concavity_se' 'num__concave points_se'
 'num__symmetry_se' 'num__fractal_dimension_se' 'num__radius_worst'
 'num__texture_worst' 'num__perimeter_worst' 'num__area_worst'
 'num__smoothness_worst' 'num__compactness_worst' 'num__concavity_worst'
 'num__concave points_worst' 'num__symmetry_worst'
 'num__fractal_dimension_worst']


Top fitur: [('num__concave points_worst', np.float64(733.7249325739482)), ('num__perimeter_worst', np.float64(717.2464871041376)), ('num__radius_worst', np.float64(692.861395260564)), ('num__concave points_mean', np.float64(684.5268452011383)), ('num__perimeter_mean', np.float64(548.4132358502825)), ('num__a